# Analyse de paroles de chansons
---

Nous avons récupérer la quasi-totalité des paroles de plusieurs groupes depuis le site genius lyrics. Les paroles sont toutes les paroles d'une sélection de groupe qu'aime bien notre utilisateur. Nous allons analyser la qualité des données et voir ce qu'il est possible d'en déduire sur l'utilisateur.

## Import des bibliothèques

In [ ]:
from langdetect import detect
import pandas as pd
from profanity_check import predict

In [ ]:
all_songs_df = pd.read_csv("all_songs.csv", sep=";")

In [ ]:
all_songs_df["release_date"] = pd.to_datetime(all_songs_df["release_date"])

## Analyse de la qualité
---
 Nous allons voir s'il n'existe pas des chansons manquantes, vérifier la langue, quelles sont les dates des chansons et analyser le nombre de vues.

Analyse des colonnes numériques

In [ ]:
# Apporte peu d'informations et oublie le fait que des valeurs sont vides
# On constate que seules les chansons avec des paroles récoltant de 5000 vues possèdent le nombre de vues (on remarque cela en regardant le min)
# Les chansons sont classées par ordre de popularité (déduisible en regardant les valeurs de lyrics-view).
all_songs_df.describe()

,release_date,lyrics_view
count,3236,1.642000e+03
mean,2013-06-06 05:32:51.322620416,3.749365e+04
min,1993-12-14 00:00:00,5.001000e+03
25%,2008-06-24 00:00:00,9.085500e+03
50%,2013-11-22 00:00:00,1.615300e+04
75%,2018-10-19 00:00:00,3.423250e+04
max,2023-12-29 00:00:00,1.179103e+06
std,NaN,6.787042e+04


In [ ]:
# On construit la colonne "notoriety" quelles sont les titres les plus consultés du groupe
all_songs_df["notoriety"] = all_songs_df.groupby("band_name")["song_title"].transform('cumcount')
print(f"Sur notre jeu de données, {round(all_songs_df['lyrics_view'].isna().sum() / len(all_songs_df) * 100, 2)}% des chansons ne possèdent pas le nombre de vues.")

Sur notre jeu de données, 56.81% des chansons ne possèdent pas le nombre de vues.


Analyse des valeurs manquantes

In [ ]:
# On constate qu'on dispose de paroles pour la plupart des chansons. On regarde rapidement celles dont les paroles sont manquantes.
all_songs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3802 entries, 0 to 3801
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   band_name                 3802 non-null   object        
 1   song_title                3802 non-null   object        
 2   album_name                3574 non-null   object        
 3   lyrics                    3709 non-null   object        
 4   release_date              3236 non-null   datetime64[ns]
 5   release_date_for_display  3330 non-null   object        
 6   lyrics_view               1642 non-null   float64       
 7   feat                      3802 non-null   object        
 8   is_feat                   3802 non-null   bool          
 9   primary_artist_name       3802 non-null   object        
 10  notoriety                 3802 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 300.9

In [ ]:
# En regardant les morceaux sans paroles, on se rend compte qu'il s'agit de morceaux instrumentaux. Il est normal qu'il n'y ait pas de paroles.
all_songs_df.loc[all_songs_df["lyrics"].isna()].tail(30)

,band_name,song_title,album_name,lyrics,release_date,release_date_for_display,lyrics_view,feat,is_feat,primary_artist_name,notoriety
2489,Nightwish,All The Works Of Nature Which Adorn The World ...,HUMAN. :||: NATURE.,NaN,2020-04-10,"April 10, 2020",NaN,[],False,Nightwish,95
2492,Nightwish,All The Works Of Nature Which Adorn The World ...,HUMAN. :||: NATURE.,NaN,2020-04-10,"April 10, 2020",NaN,[],False,Nightwish,98
2528,Nightwish,Crimson Tide / Deep Blue Sea,From Wishes to Eternity,NaN,NaT,NaN,NaN,[],False,Nightwish,134
2529,Nightwish,Elvenjig,Decades - Live In Buenos Aires,NaN,NaT,NaN,NaN,[],False,Nightwish,135
2636,Poets of the Fall,Rogue,Jealous Gods,NaN,2014-09-19,"September 19, 2014",NaN,[],False,Poets of the Fall,101
2666,Prime Circle,Intro [All Or Nothing],All or Nothing,NaN,2008-06-24,"June 24, 2008",NaN,[],False,Prime Circle,24
2722,PVRIS,If I Don’t Wake Up Tomorrow,NaN,NaN,NaT,NaN,NaN,[],False,PVRIS,55
2727,PVRIS,Lung,NaN,NaN,2017-08-04,"August 4, 2017",NaN,[],False,PVRIS,60
2903,STARSET,UNVEILING THE ARCHITECTURE,HORIZONS,NaN,2021-10-22,"October 22, 2021",NaN,[],False,STARSET,57
2987,Steam Powered Giraffe,Heist Ho! (SteamWorld Heist Theme),Music from SteamWorld Heist,NaN,NaT,NaN,NaN,[],False,Steam Powered Giraffe,79


Analyse de la langue

In [ ]:
def identify_language(x):
    try:
        return(detect(x))
    except:
        return("language_error")

all_songs_df["text_language"] = all_songs_df["lyrics"].apply(identify_language)
all_songs_df.loc[all_songs_df["lyrics"].isna(), "text_language"] = "no_lyrics"

In [ ]:
# On constate que la majorité des paroles sont en anglais.
all_songs_df.groupby("text_language").count()["band_name"]

text_language
af              1
en           3704
et              2
fr              1
no_lyrics      93
sw              1
Name: band_name, dtype: int64

In [ ]:
# Sur les chansons notées comme n'étant pas en anglais, on constate qu'elles sont en anglais et il y en a une qui semble fausse (Tied up de PVRIS). On la supprime.
all_songs_df.loc[~all_songs_df["text_language"].isin(["en", "no_lyrics"])]

,band_name,song_title,album_name,lyrics,release_date,release_date_for_display,lyrics_view,feat,is_feat,primary_artist_name,notoriety,text_language
1552,Foals,Providence,Holy Fire,I know I cannot be true\nI'm an animal just li...,2013-02-08,"February 8, 2013",6458.0,[],False,Foals,53,et
2132,Lord of the Lost,Digging Deeper,Judas (Deluxe Edition),I'm digging in deeper\nI'm digging deeper\nI'm...,2021-07-02,"July 2, 2021",NaN,"[{'api_path': '/artists/1702821', 'header_imag...",True,Lord of the Lost,195,af
2171,Metric,Poster of a Girl,Live It Out,\nCan't stand by myself\nHate to sleep alone\n...,2006-05-15,"May 15, 2006",13104.0,[],False,Metric,11,fr
2270,Metric,I Don’t Wanna Wake Up,Cosmopolis (Original Motion Picture Soundtrack),(Heavily distorted vocals)\nI don't wanna wake...,NaT,2012,NaN,[],False,Metric,110,sw
2729,PVRIS,Tied Up,NaN,**soon**You might also likeEmbed,NaT,NaN,NaN,[],False,PVRIS,62,et


In [ ]:
all_songs_df = all_songs_df.loc[~((all_songs_df["song_title"] == "Tied Up") & (all_songs_df["band_name"] == "PVRIS"))]

Analyse des doublons

In [ ]:
# On constate que tous les couples groupes / titres sont uniques. On ne devrait donc pas posséder de doublons.
all_songs_df.groupby(["band_name", "song_title"])["text_language"].count().sort_values(ascending=False)

band_name          song_title                                 
Amaranthe          1.000.000 Lightyears                           1
PVRIS              Bad World                                      1
Nightwish          White Night Fantasy                            1
                   Whoever Brings the Night                       1
                   Wish I Had an Angel                            1
                                                                 ..
Deftones           Pink Maggit                                    1
                   Pittura Infamante                              1
                   Please Please Please Let Me Get What I Want    1
                   Poltergeist                                    1
Within Temptation  Worth Dying For                                1
Name: text_language, Length: 3801, dtype: int64

In [ ]:
all_songs_df.to_csv("all_songs_df.csv", index=False, sep=";")

# Analyse des données
---

Toutes les chansons reflètent les goûts d'un utilisateur. Que peut-on dire des groupes puis de cet utilisateur ?

Afin de pouvoir faire notre analyse, nous allons construire un dataset des groupes qui inclue le nombre de chansons, nombre d'albums, nombre maximal de vues, date premier album et dernier album...

In [ ]:
group_df = pd.DataFrame(all_songs_df.groupby("band_name")["song_title"].count())

Nombre d'albums et d'EP

In [ ]:
# On se rend compte que le nombre d'albums ne sera pas facile à obtenir car on possède des versions alternatives d'un même album et des singles
all_songs_df.dropna(subset="album_name").groupby("band_name")["album_name"].unique().iloc[0]

array(['Helix', 'Amaranthe', 'Massive Addictive', 'Manifest', 'The Nexus',
       'Maximalism', 'PvP (Remix) - Single', ' Manifest (Bonus Version) ',
       'The Catalyst', 'Communio Lupatum'], dtype=object)

In [ ]:
group_df["album_list"] = all_songs_df.dropna(subset="album_name").groupby("band_name")["album_name"].unique()
group_df["num_album"] = group_df["album_list"].apply(lambda x : len([title for title in x if (("live at" not in title.lower()) & ("version" not in title.lower()) & ("single" not in title.lower()) & ("session" not in title.lower()) & ("edition" not in title.lower()))]))

Plus petite et plus grande date de publication

In [ ]:
group_df["smallest_date"] = all_songs_df.groupby("band_name")["release_date"].min()
group_df["biggest_date"] = all_songs_df.groupby("band_name")["release_date"].max()

Nombre maximal de vues

In [ ]:
group_df["max_view"] = all_songs_df.groupby("band_name")["lyrics_view"].max()

Nombre de feats

In [ ]:
group_df["num_feat"] = all_songs_df.groupby("band_name")["is_feat"].sum()

Vérification de la vulgarité

Quels sont les groupes les plus vulgaires ?

In [ ]:
all_songs_df["has_profanity"] = all_songs_df["lyrics"].fillna("").apply(lambda x : predict([x])[0])

In [ ]:
all_songs_df.groupby("has_profanity")["band_name"].count()

has_profanity
0    3256
1     545
Name: band_name, dtype: int64

In [ ]:
group_df["profanity_count"] = all_songs_df.groupby("band_name")["has_profanity"].sum().sort_values(ascending=False)
group_df["profanity_ratio"] = group_df["profanity_count"] / group_df["song_title"] 

In [ ]:
group_df

,song_title,album_list,num_album,smallest_date,biggest_date,max_view,num_feat,profanity_count,profanity_ratio
band_name,,,,,,,,,
Amaranthe,83,"[Helix, Amaranthe, Massive Addictive, Manifest...",8,2009-02-18,2023-11-16,11113.0,5,4,0.048193
Anthony Green,79,"[Pixie Queen, Studio 4 Acoustic Session, Would...",9,2008-08-05,2023-05-13,28210.0,5,11,0.139241
Architects,127,"[Holy Hell, All Our Gods Have Abandoned Us, Li...",13,2006-05-15,2023-12-04,232410.0,10,33,0.259843
Bad Omens,40,"[THE DEATH OF PEACE OF MIND, Bad Omens, Findin...",4,2015-12-18,2022-02-25,97665.0,0,10,0.250000
Blue October,175,"[Foiled, History For Sale, I Hope You’re Happy...",14,2000-08-15,2023-10-13,457485.0,4,36,0.205714
Brand New,85,"[The Devil and God Are Raging Inside Me, Deja ...",9,2001-10-09,2017-08-17,303815.0,0,6,0.070588
Breaking Benjamin,80,"[Phobia, Ember, Dark Before Dawn, We Are Not A...",9,2002-08-27,2019-12-06,204551.0,2,24,0.300000
Bring Me The Horizon,125,"[Live at Wembley, POST HUMAN: SURVIVAL HORROR,...",13,2003-10-02,2023-10-13,435778.0,23,56,0.448000
CHVRCHES,76,[The Bones of What You Believe (Special Editio...,10,2012-11-05,2023-10-20,221424.0,2,7,0.092105


In [ ]:
group_df.to_csv("group_df.csv", index=True, sep=";")

## Bilan 1)
---

Que sait-on de cet utilisateur ?

La plupart des groupes datent des années 2000 / 2010 et semblent encore être en activité (au moins un album dans les cinq dernières années). En terme de vues, quelques groupes semblent inconnus (aucune chanson avec plus de 5000 vues). Pour un certain nombre de groupe, on a moins de 10 chasons par albums ce qui semble plutôt indiquer des chansons longues ou beaucoup d'EP ?